# MUFG DataScience Challenge 2025 - Exploratory Data Analysis

This notebook performs exploratory data analysis on the crowdfunding dataset to understand patterns and relationships in the data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Data Loading and Basic Inspection

In [ ]:
# Load the training data
train_df = pd.read_csv('../../data/raw/train.csv')

print(f"Dataset shape: {train_df.shape}")
print(f"\nColumn names:")
print(train_df.columns.tolist())

In [ ]:
# Display first few rows
train_df.head()

In [ ]:
# Basic info about the dataset
train_df.info()

In [ ]:
# Check data types and basic statistics
train_df.describe()

## 2. Target Variable Analysis

In [ ]:
# Target variable distribution
target_counts = train_df['final_status'].value_counts()
print("Target variable distribution:")
print(target_counts)
print(f"\nSuccess rate: {target_counts[1] / len(train_df) * 100:.2f}%")

# Visualize target distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot
target_counts.plot(kind='bar', ax=ax1, color=['coral', 'lightblue'])
ax1.set_title('Target Variable Distribution')
ax1.set_xlabel('Final Status (0=Failed, 1=Success)')
ax1.set_ylabel('Count')
ax1.tick_params(axis='x', rotation=0)

# Pie chart
ax2.pie(target_counts.values, labels=['Failed (0)', 'Success (1)'], 
        autopct='%1.1f%%', colors=['coral', 'lightblue'])
ax2.set_title('Target Variable Proportion')

plt.tight_layout()
plt.show()

## 3. Missing Values Analysis

In [ ]:
# Check for missing values
missing_data = train_df.isnull().sum()
missing_percentage = (missing_data / len(train_df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percentage
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print("Missing values summary:")
    print(missing_df)
else:
    print("No missing values found in the dataset!")

## 4. Numerical Features Analysis

In [ ]:
# Identify numerical columns
numerical_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numerical columns: {numerical_cols}")

# Statistical summary of numerical features
train_df[numerical_cols].describe()

In [ ]:
# Distribution of goal amounts
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(train_df['goal'], bins=50, alpha=0.7, color='skyblue')
plt.title('Distribution of Goal Amount')
plt.xlabel('Goal Amount')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(np.log1p(train_df['goal']), bins=50, alpha=0.7, color='lightgreen')
plt.title('Distribution of Log(Goal Amount + 1)')
plt.xlabel('Log(Goal Amount + 1)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.boxplot(train_df['goal'])
plt.title('Box Plot of Goal Amount')
plt.ylabel('Goal Amount')

plt.tight_layout()
plt.show()

In [ ]:
# Goal amount by success/failure
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
train_df.boxplot(column='goal', by='final_status', ax=plt.gca())
plt.title('Goal Amount by Final Status')
plt.suptitle('')

plt.subplot(1, 2, 2)
for status in [0, 1]:
    subset = train_df[train_df['final_status'] == status]
    plt.hist(np.log1p(subset['goal']), bins=30, alpha=0.6, 
             label=f'Status {status}', density=True)
plt.title('Log(Goal Amount) Distribution by Status')
plt.xlabel('Log(Goal Amount + 1)')
plt.ylabel('Density')
plt.legend()

plt.tight_layout()
plt.show()

# Statistical comparison
print("Goal amount statistics by final status:")
print(train_df.groupby('final_status')['goal'].describe())

## 5. Temporal Features Analysis

In [ ]:
# Convert timestamp columns to datetime
timestamp_cols = ['deadline', 'state_changed_at', 'created_at', 'launched_at']

for col in timestamp_cols:
    train_df[f'{col}_dt'] = pd.to_datetime(train_df[col], unit='s')

# Display sample of converted timestamps
print("Sample of converted timestamps:")
print(train_df[['deadline', 'deadline_dt', 'created_at', 'created_at_dt']].head())

In [ ]:
# Create derived temporal features
train_df['campaign_duration'] = (train_df['deadline_dt'] - train_df['launched_at_dt']).dt.days
train_df['prep_time'] = (train_df['launched_at_dt'] - train_df['created_at_dt']).dt.days

# Extract date components
train_df['launch_year'] = train_df['launched_at_dt'].dt.year
train_df['launch_month'] = train_df['launched_at_dt'].dt.month
train_df['launch_day_of_week'] = train_df['launched_at_dt'].dt.dayofweek

print("Sample of derived temporal features:")
print(train_df[['campaign_duration', 'prep_time', 'launch_year', 'launch_month']].head())

In [ ]:
# Analyze temporal patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Campaign duration distribution
axes[0,0].hist(train_df['campaign_duration'], bins=50, alpha=0.7, color='lightcoral')
axes[0,0].set_title('Campaign Duration Distribution')
axes[0,0].set_xlabel('Duration (days)')

# Success rate by launch year
yearly_success = train_df.groupby('launch_year')['final_status'].agg(['count', 'mean']).reset_index()
axes[0,1].bar(yearly_success['launch_year'], yearly_success['mean'], alpha=0.7, color='lightblue')
axes[0,1].set_title('Success Rate by Launch Year')
axes[0,1].set_xlabel('Year')
axes[0,1].set_ylabel('Success Rate')

# Success rate by launch month
monthly_success = train_df.groupby('launch_month')['final_status'].mean()
axes[1,0].bar(monthly_success.index, monthly_success.values, alpha=0.7, color='lightgreen')
axes[1,0].set_title('Success Rate by Launch Month')
axes[1,0].set_xlabel('Month')
axes[1,0].set_ylabel('Success Rate')

# Success rate by day of week
dow_success = train_df.groupby('launch_day_of_week')['final_status'].mean()
dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1,1].bar(range(7), dow_success.values, alpha=0.7, color='gold')
axes[1,1].set_title('Success Rate by Day of Week')
axes[1,1].set_xlabel('Day of Week')
axes[1,1].set_ylabel('Success Rate')
axes[1,1].set_xticks(range(7))
axes[1,1].set_xticklabels(dow_labels)

plt.tight_layout()
plt.show()

In [ ]:
## 5.1. Advanced Temporal Analysis - Preparation & Campaign Timing

Deep dive into temporal patterns focusing on preparation time, campaign duration, and state change timing patterns.

In [ ]:
# Create advanced temporal features
def create_advanced_time_features(df):
    """Create advanced temporal features with data cleaning"""
    df = df.copy()
    
    # Calculate time intervals in days
    df['preparation_days'] = (df['launched_at'] - df['created_at']) / (24*3600)
    df['campaign_days'] = (df['deadline'] - df['launched_at']) / (24*3600)  
    df['state_change_days'] = (df['state_changed_at'] - df['launched_at']) / (24*3600)
    
    # Calculate state change ratio (when during campaign the state changed)
    df['state_change_ratio'] = df['state_change_days'] / df['campaign_days']
    
    return df

# Apply advanced feature engineering
print("Creating advanced temporal features...")
train_df = create_advanced_time_features(train_df)

# Check for data quality issues (negative time intervals)
print("\n=== DATA QUALITY CHECK ===")
print("Checking for negative time intervals...")

negative_prep = (train_df['preparation_days'] < 0).sum()
negative_campaign = (train_df['campaign_days'] < 0).sum()

print(f"Projects with negative preparation time: {negative_prep}")
print(f"Projects with negative campaign duration: {negative_campaign}")

# Remove projects with negative time intervals
original_size = len(train_df)
train_df = train_df[
    (train_df['preparation_days'] >= 0) & 
    (train_df['campaign_days'] >= 0)
].copy()

eliminated_count = original_size - len(train_df)
print(f"\n📊 ELIMINATED {eliminated_count} projects with invalid time intervals")
print(f"Remaining projects: {len(train_df):,} (was {original_size:,})")

In [ ]:
# Statistical analysis of temporal features
print("=== STATISTICAL ANALYSIS OF TEMPORAL FEATURES ===")

temporal_features = ['preparation_days', 'campaign_days', 'state_change_days', 'state_change_ratio']

print("\nOverall Statistics:")
stats_df = train_df[temporal_features].describe()
print(stats_df.round(2))

print("\n" + "="*60)
print("TEMPORAL FEATURES BY SUCCESS STATUS")
print("="*60)

# Analysis by success status
for feature in ['preparation_days', 'campaign_days']:
    print(f"\n📈 {feature.upper().replace('_', ' ')} ANALYSIS:")
    
    # Statistics by success status
    success_stats = train_df.groupby('final_status')[feature].describe().round(2)
    print(success_stats)
    
    # Success rate by quartiles
    quartiles = pd.qcut(train_df[feature], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    quartile_success = train_df.groupby(quartiles)['final_status'].agg(['count', 'mean']).round(3)
    quartile_success.columns = ['Project_Count', 'Success_Rate']
    print(f"\nSuccess rate by {feature} quartiles:")
    print(quartile_success)
    print("-" * 40)

In [ ]:
# Investigate state_changed_at mystery
print("=== STATE_CHANGED_AT MYSTERY INVESTIGATION ===")

# Check if state_changed_at equals deadline for failed projects
failed_projects = train_df[train_df['final_status'] == 0]
successful_projects = train_df[train_df['final_status'] == 1]

# Check state_changed_at == deadline
failed_state_eq_deadline = (failed_projects['state_changed_at'] == failed_projects['deadline']).sum()
successful_state_eq_deadline = (successful_projects['state_changed_at'] == successful_projects['deadline']).sum()

print(f"\n🔍 HYPOTHESIS: state_changed_at == deadline for failed projects")
print(f"Failed projects where state_changed_at == deadline: {failed_state_eq_deadline:,} / {len(failed_projects):,} ({failed_state_eq_deadline/len(failed_projects)*100:.1f}%)")
print(f"Successful projects where state_changed_at == deadline: {successful_state_eq_deadline:,} / {len(successful_projects):,} ({successful_state_eq_deadline/len(successful_projects)*100:.1f}%)")

# Analyze state change timing patterns
print(f"\n📊 STATE CHANGE TIMING PATTERNS:")
print(f"Failed projects - Average state change ratio: {failed_projects['state_change_ratio'].mean():.3f}")
print(f"Successful projects - Average state change ratio: {successful_projects['state_change_ratio'].mean():.3f}")

# Check for early vs late state changes
early_change = train_df[train_df['state_change_ratio'] <= 0.5]
late_change = train_df[train_df['state_change_ratio'] > 0.5]

print(f"\n⏰ EARLY vs LATE STATE CHANGES:")
print(f"Early changes (≤50% of campaign): {len(early_change):,} projects, Success rate: {early_change['final_status'].mean():.3f}")
print(f"Late changes (>50% of campaign): {len(late_change):,} projects, Success rate: {late_change['final_status'].mean():.3f}")

# Special cases analysis
state_change_at_launch = (train_df['state_change_days'] == 0).sum()
state_change_after_deadline = (train_df['state_change_days'] > train_df['campaign_days']).sum()

print(f"\n🚨 SPECIAL CASES:")
print(f"State changed at launch (day 0): {state_change_at_launch:,} projects")
print(f"State changed after deadline: {state_change_after_deadline:,} projects")

In [ ]:
# Visualize temporal patterns and success rates
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Preparation days distribution
axes[0,0].hist(train_df['preparation_days'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Preparation Days Distribution')
axes[0,0].set_xlabel('Preparation Days')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(train_df['preparation_days'].median(), color='red', linestyle='--', label=f'Median: {train_df["preparation_days"].median():.1f}')
axes[0,0].legend()

# 2. Campaign days distribution  
axes[0,1].hist(train_df['campaign_days'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].set_title('Campaign Days Distribution')
axes[0,1].set_xlabel('Campaign Days')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(train_df['campaign_days'].median(), color='red', linestyle='--', label=f'Median: {train_df["campaign_days"].median():.1f}')
axes[0,1].legend()

# 3. State change ratio distribution
axes[0,2].hist(train_df['state_change_ratio'], bins=50, alpha=0.7, color='coral', edgecolor='black')
axes[0,2].set_title('State Change Ratio Distribution')
axes[0,2].set_xlabel('State Change Ratio (0=start, 1=end)')
axes[0,2].set_ylabel('Frequency')
axes[0,2].axvline(train_df['state_change_ratio'].median(), color='red', linestyle='--', label=f'Median: {train_df["state_change_ratio"].median():.2f}')
axes[0,2].legend()

# 4. Success rate by preparation days (binned)
prep_bins = pd.cut(train_df['preparation_days'], bins=15)
prep_success = train_df.groupby(prep_bins)['final_status'].agg(['mean', 'count'])
prep_success = prep_success[prep_success['count'] >= 50]  # Only bins with sufficient data

x_prep = range(len(prep_success))
axes[1,0].bar(x_prep, prep_success['mean'], alpha=0.7, color='skyblue')
axes[1,0].set_title('Success Rate by Preparation Days')
axes[1,0].set_xlabel('Preparation Days (binned)')
axes[1,0].set_ylabel('Success Rate')
axes[1,0].set_xticks(x_prep[::2])  # Show every 2nd label to avoid crowding
axes[1,0].tick_params(axis='x', rotation=45)

# 5. Success rate by campaign days (binned)
campaign_bins = pd.cut(train_df['campaign_days'], bins=15)
campaign_success = train_df.groupby(campaign_bins)['final_status'].agg(['mean', 'count'])
campaign_success = campaign_success[campaign_success['count'] >= 50]  # Only bins with sufficient data

x_campaign = range(len(campaign_success))
axes[1,1].bar(x_campaign, campaign_success['mean'], alpha=0.7, color='lightgreen')
axes[1,1].set_title('Success Rate by Campaign Days')
axes[1,1].set_xlabel('Campaign Days (binned)')
axes[1,1].set_ylabel('Success Rate')
axes[1,1].set_xticks(x_campaign[::2])
axes[1,1].tick_params(axis='x', rotation=45)

# 6. Success rate by state change timing
state_bins = pd.cut(train_df['state_change_ratio'], bins=10)
state_success = train_df.groupby(state_bins)['final_status'].agg(['mean', 'count'])
state_success = state_success[state_success['count'] >= 30]

x_state = range(len(state_success))
axes[1,2].bar(x_state, state_success['mean'], alpha=0.7, color='coral')
axes[1,2].set_title('Success Rate by State Change Timing')
axes[1,2].set_xlabel('State Change Ratio (binned)')
axes[1,2].set_ylabel('Success Rate')
axes[1,2].set_xticks(x_state)
axes[1,2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Advanced temporal insights summary
print("=" * 70)
print("🕰️  ADVANCED TEMPORAL ANALYSIS - KEY FINDINGS")
print("=" * 70)

# Optimal ranges analysis
prep_quartile_success = train_df.groupby(pd.qcut(train_df['preparation_days'], q=4))['final_status'].mean()
campaign_quartile_success = train_df.groupby(pd.qcut(train_df['campaign_days'], q=4))['final_status'].mean()

best_prep_quartile = prep_quartile_success.idxmax()
best_campaign_quartile = campaign_quartile_success.idxmax()

print(f"\n📈 OPTIMAL TIMING PATTERNS:")
print(f"   Best preparation quartile: {best_prep_quartile} (Success rate: {prep_quartile_success.max():.3f})")
print(f"   Best campaign quartile: {best_campaign_quartile} (Success rate: {campaign_quartile_success.max():.3f})")

# Extreme cases analysis
very_short_prep = train_df[train_df['preparation_days'] <= 1]['final_status'].mean()
very_long_prep = train_df[train_df['preparation_days'] >= 180]['final_status'].mean()
very_short_campaign = train_df[train_df['campaign_days'] <= 15]['final_status'].mean()
very_long_campaign = train_df[train_df['campaign_days'] >= 90]['final_status'].mean()

print(f"\n⚠️  EXTREME CASES ANALYSIS:")
print(f"   Very short prep (≤1 day): {very_short_prep:.3f} success rate")
print(f"   Very long prep (≥180 days): {very_long_prep:.3f} success rate")
print(f"   Very short campaign (≤15 days): {very_short_campaign:.3f} success rate")
print(f"   Very long campaign (≥90 days): {very_long_campaign:.3f} success rate")

# State change insights
failed_at_deadline_pct = failed_state_eq_deadline / len(failed_projects) * 100
successful_at_deadline_pct = successful_state_eq_deadline / len(successful_projects) * 100

print(f"\n🔍 STATE_CHANGED_AT MYSTERY SOLVED:")
print(f"   Failed projects ending at deadline: {failed_at_deadline_pct:.1f}%")
print(f"   Successful projects ending at deadline: {successful_at_deadline_pct:.1f}%")
print(f"   → Hypothesis CONFIRMED: Failed projects often end exactly at deadline")

print(f"\n💡 FEATURE ENGINEERING RECOMMENDATIONS:")
print(f"   1. Create preparation_days categories (short/medium/long)")
print(f"   2. Create campaign_days categories (optimal 30-60 day range)")
print(f"   3. Use state_change_ratio as success predictor")
print(f"   4. Flag projects ending exactly at deadline (failure indicator)")
print(f"   5. Create 'rushed' projects feature (very short prep time)")

print(f"\n✅ Analysis completed with {len(train_df):,} projects")

## 6. Categorical Features Analysis

In [ ]:
# Identify categorical columns
categorical_cols = ['country', 'currency', 'disable_communication']

# Analyze country distribution
country_stats = train_df.groupby('country').agg({
    'final_status': ['count', 'mean']
}).round(3)
country_stats.columns = ['Project_Count', 'Success_Rate']
country_stats = country_stats.sort_values('Project_Count', ascending=False)

print("Top 15 countries by project count:")
print(country_stats.head(15))

In [ ]:
# Visualize country analysis
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Top countries by project count
top_countries = country_stats.head(10)
axes[0].bar(range(len(top_countries)), top_countries['Project_Count'], alpha=0.7)
axes[0].set_title('Top 10 Countries by Project Count')
axes[0].set_xlabel('Country')
axes[0].set_ylabel('Project Count')
axes[0].set_xticks(range(len(top_countries)))
axes[0].set_xticklabels(top_countries.index, rotation=45)

# Success rate for top countries
axes[1].bar(range(len(top_countries)), top_countries['Success_Rate'], alpha=0.7, color='orange')
axes[1].set_title('Success Rate for Top 10 Countries')
axes[1].set_xlabel('Country')
axes[1].set_ylabel('Success Rate')
axes[1].set_xticks(range(len(top_countries)))
axes[1].set_xticklabels(top_countries.index, rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze currency distribution
currency_stats = train_df.groupby('currency').agg({
    'final_status': ['count', 'mean']
}).round(3)
currency_stats.columns = ['Project_Count', 'Success_Rate']
currency_stats = currency_stats.sort_values('Project_Count', ascending=False)

print("Currency distribution:")
print(currency_stats.head(10))

# Analyze disable_communication feature
comm_stats = train_df.groupby('disable_communication')['final_status'].agg(['count', 'mean'])
print("\nCommunication setting analysis:")
print(comm_stats)

## 7. Text Features Analysis

In [ ]:
# Analyze text features: name, desc, keywords
text_cols = ['name', 'desc', 'keywords']

# Calculate text lengths
for col in text_cols:
    train_df[f'{col}_length'] = train_df[col].str.len()
    train_df[f'{col}_word_count'] = train_df[col].str.split().str.len()

# Analyze text statistics by success
text_stats = train_df.groupby('final_status')[['name_length', 'desc_length', 'keywords_length']].mean()
print("Average text lengths by success status:")
print(text_stats)

In [ ]:
# Visualize text length distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(['name_length', 'desc_length', 'keywords_length']):
    for status in [0, 1]:
        subset = train_df[train_df['final_status'] == status]
        axes[i].hist(subset[col], bins=30, alpha=0.6, label=f'Status {status}', density=True)
    
    axes[i].set_title(f'{col.replace("_", " ").title()} Distribution')
    axes[i].set_xlabel('Length')
    axes[i].set_ylabel('Density')
    axes[i].legend()

plt.tight_layout()
plt.show()

## 8. Correlation Analysis

In [ ]:
# Select numerical features for correlation analysis
corr_features = ['goal', 'final_status', 'campaign_duration', 'prep_time', 
                'launch_year', 'launch_month', 'launch_day_of_week',
                'name_length', 'desc_length', 'keywords_length']

# Calculate correlation matrix
corr_matrix = train_df[corr_features].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Show correlations with target variable
target_corr = corr_matrix['final_status'].sort_values(ascending=False)
print("Correlations with target variable (final_status):")
print(target_corr)

## 9. Key Insights Summary

In [ ]:
print("=== KEY INSIGHTS FROM EDA ===")
print(f"\n1. Dataset Overview:")
print(f"   - Total projects: {len(train_df):,}")
print(f"   - Success rate: {train_df['final_status'].mean()*100:.1f}%")
print(f"   - Features: {train_df.shape[1]} columns")

print(f"\n2. Target Variable:")
success_rate = train_df['final_status'].mean() * 100
if success_rate < 40:
    balance_note = "Highly imbalanced - consider sampling techniques"
elif success_rate < 45:
    balance_note = "Moderately imbalanced"
else:
    balance_note = "Relatively balanced"
print(f"   - {balance_note}")

print(f"\n3. Goal Amount:")
goal_stats = train_df['goal'].describe()
print(f"   - Median goal: ${goal_stats['50%']:,.0f}")
print(f"   - Mean goal: ${goal_stats['mean']:,.0f}")
print(f"   - High variance (skewed distribution)")

print(f"\n4. Temporal Patterns:")
print(f"   - Average campaign duration: {train_df['campaign_duration'].mean():.0f} days")
print(f"   - Launch years: {train_df['launch_year'].min()}-{train_df['launch_year'].max()}")

print(f"\n5. Geographic Distribution:")
top_country = country_stats.index[0]
top_country_pct = (country_stats.iloc[0]['Project_Count'] / len(train_df)) * 100
print(f"   - Top country: {top_country} ({top_country_pct:.1f}% of projects)")
print(f"   - Number of countries: {train_df['country'].nunique()}")

print(f"\n6. Strongest Predictors (correlation with success):")
top_corr = target_corr.drop('final_status').head(3)
for feature, corr in top_corr.items():
    print(f"   - {feature}: {corr:.3f}")